# preparation

In [ ]:
# automatic reload of modules in case of changes
%load_ext autoreload
%autoreload 2

## importing plotly

In [ ]:
import plotly
plotly.__version__

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
# using plotly in offline mode is a requirement 
# in interactive mode - too slow otherwise
import plotly.offline as pyoff

pyoff.init_notebook_mode()

In [ ]:
# interactive notebook widgets 
from ipywidgets import interact, fixed, IntSlider, Dropdown

***

# our code

In [ ]:
# to extract data from a RSSI file
from rssi import read_rssi

In [ ]:
# convert data into x, y and z
from r2labmap import dict_to_xyzt

# heatmap version 1

Exploring a single `RSSI.txt` file

In [ ]:
def heatmap(filename, sender, rssi_rank):
    X, Y, Z, T = dict_to_xyzt(read_rssi(filename, sender, rssi_rank))
    rssi = go.Heatmap( x=X, y=Y, z=Z, text=T)
    data = [rssi]
    layout = go.Layout(
        title="R2lab : RSSI(dBm) from {} when sender is fit{:02d} - rssi-rank={} "
              .format(filename, sender, rssi_rank))
    figure = go.Figure(data = data, layout=layout)
    pyoff.iplot(figure)

In [ ]:
interact(heatmap, 
         filename=fixed("DATA-RSSI.txt"),
         sender=IntSlider(min=1, max=37, step=1, value=21, continuous_update=False),
         rssi_rank=IntSlider(min=0, max=3, value=0))

# heatmap version 2

Exploring a collection of `RSSI.txt` files; assume we have collected the following files

```
shere:logs turletti$ wc -l tra*/R*
   1369 trace-T1400-r1-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r1-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r1-a7-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r54-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r54-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T1400-r54-a7-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r1-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r1-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r1-a7-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r54-a1-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r54-a3-t1-i0.008-S64-N10/RSSI.txt
   1369 trace-T500-r54-a7-t1-i0.008-S64-N10/RSSI.txt
```

Where

* `T500` means an emission power of 5dBm
* `T1400` means an emission power of 14dBm

* r1 means PHY rate=1Mbps
* r54 means PHY rate=54Mbps

* `a1` means 1 single valid antenna - and so 2 values in RSSI.txt
* `a3` means 2 valid antennas - and so 3 values in RSSI.txt
* `a7` means 4 valid antennas - and so 4 value in RSSI.txt

* `i0` `S64` and `N10` are ping parameters that are constant across the whole experiment

* xxx Thierry 2 Thierry: xxx `t1` being a constant as well I assume...

In [ ]:
def heatmap2(sender, power, rate, antennas, rssi_rank):
    
    t_part = "T{}".format(power*100)
    r_part = "r{}".format(rate)
    # assuming antennas to be 1 3 or 7
    a_part = "a{}".format(antennas)
    filename = "trace-{t_part}-{r_part}-{a_part}-t1-i0.008-S64-N10/RSSI.txt"\
               .format(**locals())
    
    data_dict = read_rssi(filename, sender, rssi_rank)
    if not data_dict:
        return

    X, Y, Z, T = dict_to_xyzt(data_dict)
    rssi = go.Heatmap( x=X, y=Y, z=Z, text=T)
    data = [rssi]
    layout = go.Layout(
        title="R2lab : received power in dBm when sender is fit{:02d}<br>from {}"
              .format(sender, filename))
    figure = go.Figure(data = data, layout=layout)
    pyoff.iplot(figure)

In [ ]:
interact(heatmap2, 
         sender=IntSlider(min=1, max=37, step=1, value=21, continuous_update=False),
         power=Dropdown( options=[5, 14], value=5, description="emission power in dBm"),
         rate=Dropdown(options=[1, 54], value=1, description="physical channel"),
         # yeah, this is weird all right
         antennas=Dropdown(options={"one" : 1, "two": 3, "four": 7},
                           value=1,
                           description="how many antennas"),
         rssi_rank=IntSlider(min=0, max=3, value=0, 
                             description="rank - don't know how to tune with the antennas value"),
        )